In [1]:
from typing import List, Dict, Any

from datasets import load_dataset, concatenate_datasets
from datasets import DatasetDict
from transformers import AutoTokenizer
from multiprocessing import cpu_count
import random
import numpy as np

random.seed(93)
np.random.seed(93)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", add_eos_token=True)
tokenizer.bos_token

'<s>'

In [3]:
tokenizer(["The quick brown fox", "jumped over the lazy dog"])

{'input_ids': [[1, 415, 2936, 9060, 285, 1142, 2], [1, 14949, 754, 272, 17898, 3914, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [4]:
ds = load_dataset("glue", "mnli")
ds

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [5]:
ds["eval"] = concatenate_datasets([ds["validation_matched"], ds["validation_mismatched"]])
del ds["validation_matched"]
del ds["validation_mismatched"]
ds

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
    eval: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 19647
    })
})

In [6]:
ds["train"][0]["premise"]

'Conceptually cream skimming has two basic dimensions - product and geography.'

In [7]:
len(ds["train"]["premise"])

392702

In [8]:
def create_sentence_pairs(batch):
    result = {
        "input_ids": [],
        "attention_mask": []
    }
    premise_tokenized = tokenizer(batch['premise'])
    hypothesis_tokenized = tokenizer(batch['hypothesis'])

    for i in range(len(batch["premise"])):

        current_ids = []
        current_mask = []

        current_ids.extend(premise_tokenized["input_ids"][i])
        current_mask.extend(premise_tokenized["attention_mask"][i])
        current_ids.extend(hypothesis_tokenized["input_ids"][i][1:])
        current_mask.extend(hypothesis_tokenized["input_ids"][i][1:])

        result["input_ids"].append(current_ids)
        result["attention_mask"].append(current_mask)

    return result


ds = ds.map(create_sentence_pairs, batched=True, num_proc=cpu_count() - 1)
ds = ds.remove_columns(["premise", "hypothesis"])
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 392702
    })
    test_matched: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9847
    })
    eval: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 19647
    })
})

In [9]:
tokenizer.decode(ds["train"][0]["input_ids"])

'<s> Conceptually cream skimming has two basic dimensions - product and geography.</s> Product and geography are what make cream skimming work. </s>'

In [10]:
ds.save_to_disk("./data/mnli_mistral")

Saving the dataset (0/1 shards):   0%|          | 0/392702 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9796 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9847 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/19647 [00:00<?, ? examples/s]